In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import pickle
from xgboost import XGBRegressor



In [ ]:
file_path = "/content/drive/MyDrive/Data Science Projects/Cameron_headcount_prediction/headcounts.csv"

In [ ]:
cols = ['Timestamp','Date','Time','Basement_headcount','Main_floor_headcount',
                'Comments']

In [ ]:
df = pd.read_csv(file_path, names = cols)

In [ ]:
df

,Timestamp,Date,Time,Basement_headcount,Main_floor_headcount,Comments
0,Timestamp,1. Date,2. Time,3. Cameron Basement: Enter the number of people,4. Cameron Main Floor: Enter the number of pe...,5. Anything to note? Please share with Library...
1,"Monday, May 8, 2023 at 6:33:58 PM",5/8/2023,6:30 pm,17,3,Patrolled Cameron Library main floor and basem...
2,"Monday, May 8, 2023 at 7:38:16 PM",5/8/2023,7:30 pm,7,1,Patrolled Cameron Library main floor and basem...
3,"Monday, May 8, 2023 at 8:31:10 PM",5/8/2023,8:30 pm,5,0,Patrolled Cameron library main floor and basem...
4,"Monday, May 8, 2023 at 9:34:52 PM",5/8/2023,9:30 pm,2,0,Patrolled Cameron Library Library main floor a...
...,...,...,...,...,...,...
124,6/5/2023 21:32:55,6/5/2023,9:30 pm,13,4,Patrolled Cameron library main floor and basem...
125,6/6/2023 18:37:28,6/6/2023,6:30 pm,21,8,Patrolled Cameron library main floor and basem...
126,6/6/2023 19:31:35,6/6/2023,7:30 pm,19,9,Patrolled Cameron library main floor and basem...
127,6/6/2023 20:34:41,6/6/2023,8:30 pm,16,5,Patrolled Cameron library main floor and basem...


In [ ]:
data = df.copy()

In [ ]:
for col in data.columns:
    print(col)

Timestamp
Date
Time
Basement_headcount
Main_floor_headcount
Comments


In [ ]:
data = data.drop([0])

In [ ]:
data=data.drop("Comments", axis = 1)

In [ ]:
data

,Timestamp,Date,Time,Basement_headcount,Main_floor_headcount
1,"Monday, May 8, 2023 at 6:33:58 PM",5/8/2023,6:30 pm,17,3
2,"Monday, May 8, 2023 at 7:38:16 PM",5/8/2023,7:30 pm,7,1
3,"Monday, May 8, 2023 at 8:31:10 PM",5/8/2023,8:30 pm,5,0
4,"Monday, May 8, 2023 at 9:34:52 PM",5/8/2023,9:30 pm,2,0
5,"Tuesday, May 9, 2023 at 6:31:10 PM",5/9/2023,6:30 pm,13,8
...,...,...,...,...,...
124,6/5/2023 21:32:55,6/5/2023,9:30 pm,13,4
125,6/6/2023 18:37:28,6/6/2023,6:30 pm,21,8
126,6/6/2023 19:31:35,6/6/2023,7:30 pm,19,9
127,6/6/2023 20:34:41,6/6/2023,8:30 pm,16,5


In [ ]:
data.isnull().sum()

Timestamp               0
Date                    0
Time                    0
Basement_headcount      0
Main_floor_headcount    0
dtype: int64

In [ ]:
data.describe()

,Timestamp,Date,Time,Basement_headcount,Main_floor_headcount
count,128,128,128,128,128
unique,128,31,5,33,24
top,"Monday, May 8, 2023 at 6:33:58 PM",5/21/2023,6:30 pm,13,3
freq,1,5,30,9,16


In [ ]:
data= data.drop("Timestamp", axis = 1)

In [ ]:
data

,Date,Time,Basement_headcount,Main_floor_headcount
1,5/8/2023,6:30 pm,17,3
2,5/8/2023,7:30 pm,7,1
3,5/8/2023,8:30 pm,5,0
4,5/8/2023,9:30 pm,2,0
5,5/9/2023,6:30 pm,13,8
...,...,...,...,...
124,6/5/2023,9:30 pm,13,4
125,6/6/2023,6:30 pm,21,8
126,6/6/2023,7:30 pm,19,9
127,6/6/2023,8:30 pm,16,5


In [ ]:
data['Date']= pd.to_datetime(data["Date"])

In [ ]:
data

,Date,Time,Basement_headcount,Main_floor_headcount
1,2023-05-08,6:30 pm,17,3
2,2023-05-08,7:30 pm,7,1
3,2023-05-08,8:30 pm,5,0
4,2023-05-08,9:30 pm,2,0
5,2023-05-09,6:30 pm,13,8
...,...,...,...,...
124,2023-06-05,9:30 pm,13,4
125,2023-06-06,6:30 pm,21,8
126,2023-06-06,7:30 pm,19,9
127,2023-06-06,8:30 pm,16,5


In [ ]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data["Day"] = data["Date"].dt.day


In [ ]:
data

,Date,Time,Basement_headcount,Main_floor_headcount,Year,Month,Day
1,2023-05-08,6:30 pm,17,3,2023,5,8
2,2023-05-08,7:30 pm,7,1,2023,5,8
3,2023-05-08,8:30 pm,5,0,2023,5,8
4,2023-05-08,9:30 pm,2,0,2023,5,8
5,2023-05-09,6:30 pm,13,8,2023,5,9
...,...,...,...,...,...,...,...
124,2023-06-05,9:30 pm,13,4,2023,6,5
125,2023-06-06,6:30 pm,21,8,2023,6,6
126,2023-06-06,7:30 pm,19,9,2023,6,6
127,2023-06-06,8:30 pm,16,5,2023,6,6


In [ ]:
new_cols = ['Month','Day','Year','Time','Basement_headcount','Main_floor_headcount']
data = data.reindex(columns=new_cols)

In [ ]:

data

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
1,5,8,2023,6:30 pm,17,3
2,5,8,2023,7:30 pm,7,1
3,5,8,2023,8:30 pm,5,0
4,5,8,2023,9:30 pm,2,0
5,5,9,2023,6:30 pm,13,8
...,...,...,...,...,...,...
124,6,5,2023,9:30 pm,13,4
125,6,6,2023,6:30 pm,21,8
126,6,6,2023,7:30 pm,19,9
127,6,6,2023,8:30 pm,16,5


In [ ]:
data["Time"].value_counts()/len(data)

6:30 pm                           0.234375
7:30 pm                           0.226562
8:30 pm                           0.226562
9:30 pm                           0.226562
5:30 pm (Fri - Sat - Sun only)    0.085938
Name: Time, dtype: float64

In [ ]:
data["Month"].value_counts()/len(data)

5    0.773438
6    0.210938
8    0.007812
2    0.007812
Name: Month, dtype: float64

In [ ]:
data["Day"].value_counts()/len(data)

26    0.039062
28    0.039062
4     0.039062
12    0.039062
13    0.039062
14    0.039062
3     0.039062
2     0.039062
27    0.039062
19    0.039062
20    0.039062
21    0.039062
30    0.031250
29    0.031250
8     0.031250
31    0.031250
5     0.031250
1     0.031250
23    0.031250
25    0.031250
24    0.031250
9     0.031250
18    0.031250
17    0.031250
16    0.031250
15    0.031250
11    0.031250
10    0.031250
6     0.031250
Name: Day, dtype: float64

In [ ]:
data["Year"].value_counts()/len(data)

2023    1.0
Name: Year, dtype: float64

In [ ]:
data.query("Month in(2,8)")

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
6,8,9,2023,7:30 pm,9,0
57,2,20,2023,9:30 pm,3,1


In [ ]:
data.head(10)

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
1,5,8,2023,6:30 pm,17,3
2,5,8,2023,7:30 pm,7,1
3,5,8,2023,8:30 pm,5,0
4,5,8,2023,9:30 pm,2,0
5,5,9,2023,6:30 pm,13,8
6,8,9,2023,7:30 pm,9,0
7,5,9,2023,8:30 pm,4,0
8,5,9,2023,9:30 pm,2,0
9,5,10,2023,6:30 pm,15,10
10,5,10,2023,7:30 pm,17,3


In [ ]:
data[50:57]

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
51,5,19,2023,8:30 pm,10,4
52,5,19,2023,9:30 pm,5,2
53,5,20,2023,5:30 pm (Fri - Sat - Sun only),17,1
54,5,20,2023,6:30 pm,13,0
55,5,20,2023,7:30 pm,10,1
56,5,20,2023,8:30 pm,5,1
57,2,20,2023,9:30 pm,3,1


In [ ]:
data["Month"] = data["Month"].replace([8,2],5)

In [ ]:
data

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
1,5,8,2023,6:30 pm,17,3
2,5,8,2023,7:30 pm,7,1
3,5,8,2023,8:30 pm,5,0
4,5,8,2023,9:30 pm,2,0
5,5,9,2023,6:30 pm,13,8
...,...,...,...,...,...,...
124,6,5,2023,9:30 pm,13,4
125,6,6,2023,6:30 pm,21,8
126,6,6,2023,7:30 pm,19,9
127,6,6,2023,8:30 pm,16,5


In [ ]:
data["Month"].value_counts()/len(data)

5    0.789062
6    0.210938
Name: Month, dtype: float64

In [ ]:
data["Month"]= data["Month"].astype(int)
data["Day"]= data["Day"].astype(int)
data["Year"]= data["Year"].astype(int)
data["Basement_headcount"]= data["Basement_headcount"].astype(int)
data["Main_floor_headcount"]= data["Main_floor_headcount"].astype(int)

In [ ]:
data["Month"].value_counts()/len(data)

5    0.789062
6    0.210938
Name: Month, dtype: float64

In [ ]:
data

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
1,5,8,2023,6:30 pm,17,3
2,5,8,2023,7:30 pm,7,1
3,5,8,2023,8:30 pm,5,0
4,5,8,2023,9:30 pm,2,0
5,5,9,2023,6:30 pm,13,8
...,...,...,...,...,...,...
124,6,5,2023,9:30 pm,13,4
125,6,6,2023,6:30 pm,21,8
126,6,6,2023,7:30 pm,19,9
127,6,6,2023,8:30 pm,16,5


In [71]:
data["Time"] = data['Time'].map({"5:30 pm (Fri - Sat - Sun only)": "17.50", "6:30 pm":"18.50", "7:30 pm":"19.50", "8:30 pm":"20.50", "9:30 pm":"21.50"})
data["Time"]= data["Time"].astype(float)

In [ ]:
data[50:57]

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
51,5,19,2023,20.50,10,4
52,5,19,2023,21.50,5,2
53,5,20,2023,17.50,17,1
54,5,20,2023,18.50,13,0
55,5,20,2023,19.50,10,1
56,5,20,2023,20.50,5,1
57,5,20,2023,21.50,3,1


##Splitting data into train and test

In [ ]:
y = data[["Main_floor_headcount", "Basement_headcount"]]
X = data.drop(["Main_floor_headcount", "Basement_headcount"], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=42,stratify = data["Time"])

In [ ]:
X_train['Time'].value_counts() / len(X_train)

18.50    0.235294
19.50    0.225490
20.50    0.225490
21.50    0.225490
17.50    0.088235
Name: Time, dtype: float64

In [ ]:
X_test['Time'].value_counts() / len(X_test)

19.50    0.230769
18.50    0.230769
21.50    0.230769
20.50    0.230769
17.50    0.076923
Name: Time, dtype: float64

In [ ]:
X_train.head()

,Month,Day,Year,Time
80,5,27,2023,18.50
6,5,9,2023,19.50
28,5,14,2023,18.50
106,6,2,2023,17.50
118,6,4,2023,19.50


In [ ]:
y_test

,Main_floor_headcount,Basement_headcount
29,2,12
32,11,32
125,8,21
74,2,10
77,8,18
13,10,19
15,2,19
7,0,4
42,7,24
91,8,24


##Training different models to see which performs better
- First we use linear regression

In [ ]:
lin_reg = LinearRegression()

In [ ]:
lin_reg.fit(X_train, y_train)

LinearRegression()

In [ ]:
prediction = lin_reg.predict(X_test)

In [ ]:
prediction_df = pd.DataFrame(prediction, columns=['Main_floor_pred','Basement_pred'])
prediction_df

,Main_floor_pred,Basement_pred
0,6.062466,14.245217
1,8.817968,18.130646
2,11.739896,20.968445
3,3.663894,8.051567
4,8.935480,15.701101
5,7.860297,17.645352
6,2.828128,10.117141
7,2.349293,9.874494
8,4.264635,10.845083
9,9.653734,16.065072


In [ ]:
y_test

,Main_floor_headcount,Basement_headcount
29,2,12
32,11,32
125,8,21
74,2,10
77,8,18
13,10,19
15,2,19
7,0,4
42,7,24
91,8,24


##Evaulating our model using mean squared error

In [ ]:
lin_mse = mean_squared_error(y_test, prediction)

In [ ]:
lin_mse

34.492244882786096

In [ ]:
lin_rmse= np.sqrt(lin_mse)
normalized_lin_rmse = lin_rmse/(y_test.max() - y_test.min())
normalized_lin_rmse

Main_floor_headcount    0.266955
Basement_headcount      0.183532
dtype: float64

##Lets try XGBoost

,Main_floor_headcount,Basement_headcount
80,9,19
6,0,9
28,3,11
106,19,28
118,7,18
...,...,...
27,7,13
58,10,16
111,6,15
69,4,16


In [ ]:
xg_reg = XGBRegressor(random_state=42, enable)
xg_reg.fit(X_train, y_train)



ValueError: ignored

##Lets use RandomForest regression now
- We notice that this model performs the best on our data

In [ ]:
rf_reg = RandomForestRegressor(random_state =42)

In [ ]:
rf_reg.fit(X_train, y_train)
prediction_rf = rf_reg.predict(X_test)
kscores_rf = cross_val_score(rf_reg, X_test, y_test,scoring = "neg_mean_squared_error", cv= 10)
rf_rmse_kscores = np.sqrt(-kscores_rf)
rf_rmse_kscores.mean()/(y_test.max() - y_test.min())

Main_floor_headcount    0.262038
Basement_headcount      0.180151
dtype: float64

##Lets try Decision tree regrssion now


In [ ]:
dec_reg = DecisionTreeRegressor(random_state = 42)


In [ ]:
dec_reg.fit(X_train, y_train)
prediction_dc = dec_reg.predict(X_test)
kscores_dc = cross_val_score(dec_reg, X_test, y_test,scoring = "neg_mean_squared_error", cv= 10)
dc_rmse_kscores = np.sqrt(-kscores_dc)
dc_rmse_kscores.mean()/(y_test.max() - y_test.min())

Main_floor_headcount    0.334066
Basement_headcount      0.229670
dtype: float64

##Fine tuning hyperparameters using GridSeachCv


In [ ]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

In [ ]:
grid_search = GridSearchCV(rf_reg, param_grid, scoring = "neg_mean_squared_error", return_train_score=True,cv=10)
grid_search.fit(X_test, y_test)

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
#best parmeters
grid_search.best_params_

{'max_features': 4, 'n_estimators': 10}

In [ ]:
#Checking the relative importance of each feature
feature_importances = grid_search.best_estimator_.feature_importances_
attributes = list(X_train.select_dtypes(include= "number"))
sorted(zip(attributes, feature_importances), reverse = True)

[('Year', 0.0), ('Month', 0.012084181552939782), ('Day', 0.3272419679350889)]

In [ ]:
example = {"Month":[5,6],"Day": [17,2], "Year": [2023, 2023], "Time":[17.50, 18.50]}

In [ ]:
example_df = pd.DataFrame(data = example)

In [ ]:
example_df

,Month,Day,Year,Time
0,5,17,2023,17.5
1,6,2,2023,18.5


In [ ]:
rf_reg.predict(example_df)

array([[21.77, 37.31],
       [14.83, 23.44]])

In [ ]:
lin_reg.predict(example_df)

array([[11.81288811, 22.1373985 ],
       [10.78222424, 20.48315017]])

In [ ]:
data.query("Day in(2,17)")

,Month,Day,Year,Time,Basement_headcount,Main_floor_headcount
40,5,17,2023,18.50,42,26
41,5,17,2023,19.50,32,17
42,5,17,2023,20.50,24,7
43,5,17,2023,21.50,16,7
106,6,2,2023,17.50,28,19
107,6,2,2023,18.50,26,15
108,6,2,2023,19.50,12,13
109,6,2,2023,20.50,11,9
110,6,2,2023,21.50,3,3
